In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import sys, os
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt

from tqdm import tqdm
from scipy import interpolate as interp
from scipy import integrate
from scipy.optimize import differential_evolution

from spectrum import *

---

In [5]:
ld_dir = 'Primakoff_Spectra/Spectra_Primakoff_18p6/'
Ebins, prim = load_prim(ld_dir, Eoutput = True)
tbins = load_time(ld_dir)

In [6]:
# calculate the dN/dt
prim_dndt = np.zeros(prim.shape[0])

for i in range(prim.shape[0]):
    f_prim = interp.interp1d(Ebins, prim[i])
    prim_dndt[i] = integrate.quad(f_prim, 0.025, 0.1)[0]

fp_dndt = interp.interp1d(tbins, prim_dndt)
prim_N = integrate.quad(fp_dndt, 0, 10)[0]

/tmp/ipykernel_2034869/252834630.py:6: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  prim_dndt[i] = integrate.quad(f_prim, 0.025, 0.1)[0]
/tmp/ipykernel_2034869/252834630.py:9: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhap

In [15]:
prim_Ngamma = prim_N * 5.85e-7 * (1e-11)**2
print(f"N_gamma: {prim_Ngamma:.3e}")

N_gamma: 7.711e+44


In [16]:
prim_flux = prim_Ngamma / (4 * np.pi * (51 * u.kpc).to(u.cm).value**2) * 63
print(f"Flux: {prim_flux:.3e} cm^-2 s^-1")

Flux: 1.561e-01 cm^-2 s^-1


In [21]:
print(f"Limit on gagg = {(20 / prim_flux)**(1/4) * 1e-11:.2e} GeV^-1")

Limit on gagg = 3.36e-11 GeV^-1


---

# our analysis

In [22]:
from smm_analysis import *

In [28]:
def calc_smm_spec(LD_dir, smm_energy, convprob):
    '''
    Calculate the SMM spectrum for a range of ma and smm_energy.
    Parameters:
    -----------
    LD_dir: str
        Directory containing the Primakoff spectra files.
    smm_energy: array-like
        Array of SMM energies to consider in units of GeV.
    convprob: array-like
        Array of conversion probabilities as a function of energy.
    Returns:
    --------
    spec_list: list of np.ndarray
        List of spectra arrays for each SMM energy.
    '''
    Ebins, loaded = load_spec(LD_dir, Eoutput = True)
    spec_list = []
    
    for int_I in smm_energy:
        tm_spec = np.zeros(loaded.shape[0])
        
        for ti in range(loaded.shape[0]):
            photon_spec = interp.interp1d(Ebins, loaded[ti] * convprob)
            tm_spec[ti] = integrate.quad(photon_spec, *int_I)
        spec_list.append(tm_spec)
        
    return spec_list

In [29]:
calc_smm_spec(ld_dir, [[0.0041, 0.0064], [0.01, 0.025], [0.025, 0.1]], 5.85e-9)

ValueError: setting an array element with a sequence.

In [ ]:
calc_smm_spec(ld_dir, [1e-])